In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_SYNC_ACCELERATED_UPDATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### MITb Dataset

In [2]:
TRIES = 10

for it in range(TRIES):

    file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_M3500_g2o.g2o'
    data_gen.generate_SLAM_dataset_PSD(file_path=file_path)

    dataset = 'input_M3500_g2o'
    filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
    factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
    marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

    A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

    sync_convergence_threshold = 1*10**-0 # convergence threshold
    convergence_type = 'all' #all
    simulation_convergence_threshold = 1*10**-10

    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                        b, 
                                                                                                        max_iter=1_000, 
                                                                                                        mae=True if convergence_type == 'mae' else False, 
                                                                                                        convergence_threshold=simulation_convergence_threshold,
                                                                                                        show=True)
    final_mean_sync = list(mu_i)

    if iteration < 1_000:
        break
    else:
        iteration = float("inf")
        print("=========== RESTART ===========")

iteration: 1
0.015075363308670193
-----
iteration: 2
0.00019627306588941096
-----
iteration: 3
1.9016714807722025e-05
-----
iteration: 4
1.8151943396118156e-06
-----
iteration: 5
1.673265246508232e-07
-----
iteration: 6
1.316471485721616e-08
-----
iteration: 7
1.3273388905207835e-09
-----
iteration: 8
2.3283064365386963e-10
-----
iteration: 9
0.0
-----


In [3]:
print(A.shape)

(3500, 3500)


### Hardware

In [4]:
ASYNC_ITER = 10
NODE_UPDT_PE = 100
PEs = 1

async_convergence_threshold = 1*10**-5

In [5]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=False, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.006302980392860771
-----
iteration: 2
0.006302980392860771
-----
iteration: 3
0.006302980392860771
-----
iteration: 4
0.006302980392860771
-----
iteration: 5
0.006302980392860771
-----
iteration: 6
0.006302980392860771
-----
iteration: 7
0.006302980392860771
-----
iteration: 8
0.006302980392860771
-----
iteration: 9
0.006302980392860771
-----
iteration: 10
0.006302980392860771
-----
iteration: 11
0.006302980392860771
-----
iteration: 12
0.006302980392860771
-----
iteration: 13
0.006302980392860771
-----
iteration: 14
0.006302980392860771
-----
iteration: 15
0.006302980392860771
-----
iteration: 16
0.006302980392860771
-----
iteration: 17
0.006302980392860771
-----
iteration: 18
0.006235121288358579
-----
iteration: 19
0.006235121288358579
-----
iteration: 20
0.006235121288358579
-----
iteration: 21
0.006235121288358579
-----
iteration: 22
0.006235121288358579
-----
iteration: 23
0.006235121288358579
-----
iteration: 24
0.006235121288358579
-----
iteration: 25
0.006235121